In [1]:
from kinisi.diffusion_analyzer import DiffusionAnalyzer 
from pymatgen.io.vasp import Xdatcar
import scipp as sc

In [2]:
from kinisi.jump_diffusion_analyzer import JumpDiffusionAnalyzer

In [3]:
xd = Xdatcar('./example_XDATCAR.gz')

In [4]:
time_step1 = 2.0 * sc.Unit('fs')
step_skip1 = 50 * sc.Unit('dimensionless')

In [ ]:
z = JumpDiffusionAnalyzer.from_Xdatcar(xd, time_step=time_step1, 
                                       step_skip=step_skip1, specie='Li',
                                       dimension='xyz')

In [ ]:
z.mstd

In [ ]:
z.n_atoms

In [ ]:
import matplotlib.pyplot as plt


f = z.mstd.plot(marker='', norm='log', scale={'timestep': 'log'})
f.canvas.ylabel = 'MSD ' + f.canvas.ylabel
f.ax.axvline(6000)
f

In [ ]:
start_of_diffusion = 6000 * sc.Unit('fs')
z.jump_diffusion(start_of_diffusion, {'cond_max': 1e4})

In [ ]:
z.diff.covariance_matrix.plot()

In [ ]:
z.mstd

In [ ]:
f = sc.counts_to_density(z.diff.D_J.hist(samples=40), dim='samples').plot()
f.canvas.xlabel = f'D* [{z.diff.D_J.unit}]'
f.canvas.ymin = 0
f

In [ ]:
sc.mean(z.diff.intercept)

In [ ]:
sc.mean(z.diff.gradient)

In [ ]:
import numpy as np

credible_intervals = [[16, 84], [2.5, 97.5], [0.15, 99.85]]
alpha = [0.6, 0.4, 0.2]

f = z.mstd.plot(marker='', ls='-', errorbars=True, color='k')
for i, ci in enumerate(credible_intervals):     
    f.ax.fill_between(z.mstd.coords['timestep'].values,
                      *np.percentile(z.distributions, ci, axis=1),
                      alpha=alpha[i],
                      color='#0173B2',
                      lw=0)
f.canvas.ylabel = 'MSD ' + f.canvas.ylabel
f.canvas.xmin = 0
f.canvas.ymin = 0
f

In [ ]:
from corner import corner

f = corner(z.diff._flatchain)
f.show()